In [154]:
from datetime import timedelta
import warnings


import polars as pl
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import RobustScaler
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from statsmodels.tsa.arima.model import ARIMA
import joblib
import optuna
from datetime import datetime

device = "cuda" if torch.cuda.is_available() else "cpu"

warnings.filterwarnings("ignore")

In [155]:
device

'cpu'

## Test dataset

Let's calculate metrics on the first 5 minutes of the test part

In [156]:
test_df_for_net = pl.read_parquet("../data/processed/test.parquet")
print(test_df_for_net.shape)

(75570, 14)


In [157]:
test_df_for_net = test_df_for_net[:7348]
print(test_df_for_net.shape)

(7348, 14)


## Models Architectures

In [158]:
class SequenceDataset(Dataset):
    def __init__(self, df: pl.DataFrame, seq_length: int, target_col: str):
        self.seq_length = seq_length
        feature_columns = [col for col in df.columns if col != target_col]
        self.features = df.select(feature_columns).to_numpy()
        self.targets = df[target_col].to_numpy()

    def __len__(self):
        return self.features.shape[0] - self.seq_length

    def __getitem__(self, idx):
        seq_x = self.features[idx : idx + self.seq_length]
        target = self.targets[idx + self.seq_length]
        return torch.tensor(seq_x, dtype=torch.float32), torch.tensor([target], dtype=torch.float32)

In [159]:
class BasicRNN(nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        output_size,
        dropout_rate=0.5,
    ):
        super(BasicRNN, self).__init__()
        self.rnn = nn.RNN(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0,
        )
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.rnn.num_layers, x.size(0), self.rnn.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        return out


class BasicLSTM(nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        output_size,
        dropout_rate=0.5,
    ):
        super(BasicLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0,
        )
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        return out


class BasicBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_rate=0.5):
        super(BasicBiLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0,
            bidirectional=True,
        )
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.lstm.num_layers * 2, x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers * 2, x.size(0), self.lstm.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        return out


class CNNLSTM(nn.Module):
    def __init__(
        self,
        input_size,
        conv_filters=9,
        kernel_size=7,
        lstm_hidden_size=100,
        lstm_num_layers=2,
        output_size=1,
        dropout_rate=0.5,
        padding=True,
    ):
        super(CNNLSTM, self).__init__()
        self.input_size = input_size

        pad = kernel_size // 2 if padding else 0
        self.conv1d = nn.Conv1d(
            in_channels=input_size,
            out_channels=input_size * conv_filters,
            kernel_size=kernel_size,
            groups=input_size,
            padding=pad,
        )

        self.lstm = nn.LSTM(
            input_size=input_size * conv_filters,
            hidden_size=lstm_hidden_size,
            num_layers=lstm_num_layers,
            batch_first=True,
            dropout=dropout_rate,
        )
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(lstm_hidden_size, output_size)

    def forward(self, x):
        x = x.transpose(1, 2)
        conv_out = torch.tanh(self.conv1d(x))
        conv_out = conv_out.transpose(1, 2)
        batch_size = conv_out.size(0)
        h0 = torch.zeros(self.lstm.num_layers, batch_size, self.lstm.hidden_size, device=x.device)
        c0 = torch.zeros(self.lstm.num_layers, batch_size, self.lstm.hidden_size, device=x.device)
        lstm_out, _ = self.lstm(conv_out, (h0, c0))
        out = lstm_out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        return out


class AttentionLayer(nn.Module):
    def __init__(self, enc_size, dec_size, attn_hidden_size):
        super().__init__()
        self.linear_enc = nn.Linear(enc_size, attn_hidden_size)
        self.linear_dec = nn.Linear(dec_size, attn_hidden_size)
        self.linear_out = nn.Linear(attn_hidden_size, 1)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, enc, dec, inp_mask):
        batch_size, seq_length, _ = enc.shape
        query_proj = self.linear_dec(dec).unsqueeze(1)  # (batch, 1, attn_hidden_size)
        enc_proj = self.linear_enc(enc)  # (batch, seq_length, attn_hidden_size)
        energies = torch.tanh(enc_proj + query_proj)  # (batch, seq_length, attn_hidden_size)
        scores = self.linear_out(energies).squeeze(-1)  # (batch, seq_length)

        scores = scores.masked_fill(~inp_mask, -1e9)

        attn_weights = self.softmax(scores)  # (batch, seq_length)
        context = torch.sum(attn_weights.unsqueeze(-1) * enc, dim=1)  # (batch, enc_size)
        return context, attn_weights


class AttentionLSTM(nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        attn_hidden_size,
        output_size=1,
        dropout_rate=0.5,
    ):
        super(AttentionLSTM, self).__init__()
        self.encoder = nn.LSTM(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0,
        )
        self.attention = AttentionLayer(hidden_size, hidden_size, attn_hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, mask=None):
        encoder_outputs, (h_n, c_n) = self.encoder(x)
        query = h_n[-1]  # (batch, hidden_size)

        if mask is None:
            mask = torch.ones(encoder_outputs.size(0), encoder_outputs.size(1), dtype=torch.bool, device=x.device)

        context, attn_weights = self.attention(encoder_outputs, query, mask)
        out = self.dropout(context)
        out = self.fc(out)
        return out

## Methods for evaluation

In [160]:
def evaluate_model(model, dataloader, device):
    model.eval()
    predictions = []
    targets = []
    with torch.no_grad():
        for seq, target in dataloader:
            seq = seq.to(device)
            target = target.to(device)
            output = model(seq)
            predictions.append(output.cpu().numpy())
            targets.append(target.cpu().numpy())

    predictions = np.concatenate(predictions, axis=0)
    targets = np.concatenate(targets, axis=0)
    return targets, predictions


def plot_predictions(y_true, y_pred, title="Predicted vs Actual Gas Price"):
    plt.figure(figsize=(10, 5))
    plt.plot(y_true, label="Actual")
    plt.plot(y_pred, label="Predicted")
    plt.title(title)
    plt.xlabel("Time step")
    plt.ylabel("Gas Price")
    plt.legend()
    plt.show()


def calculate_metrics(y_true, y_pred):
    epsilon = 1e-8
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100
    mae = np.mean(np.abs(y_true - y_pred))
    return {"MAPE": mape, "MAE": mae}


def inverse_scale(scaled_data, scaler):
    return scaler.inverse_transform(scaled_data)

In [161]:
# scaler = joblib.load("../models/tx_scaler.pkl")
# numeric_cols = ['cost', 'gas', 'gas_fee_cap', 'gas_price']

# test_df_numeric = test_df_for_net.select(numeric_cols)
# test_df_numeric_pd = test_df_numeric.to_pandas().astype(float)

# inverted_data = inverse_scale(test_df_numeric_pd, scaler)

# test_df_for_net = test_df_for_net.with_columns(pl.Series(name='cost', values=inverted_data[:, 0]))
# test_df_for_net = test_df_for_net.with_columns(pl.Series(name='gas', values=inverted_data[:, 0]))
# test_df_for_net = test_df_for_net.with_columns(pl.Series(name='gas_fee_cap', values=inverted_data[:, 0]))
# test_df_for_net = test_df_for_net.with_columns(pl.Series(name='gas_price', values=inverted_data[:, 0]))

# test_df_for_net.head()

## Common parameters

In [162]:
seq_length = 50
target_column = "gas_price"
batch_size = 64

test_dataset = SequenceDataset(test_df_for_net, seq_length, target_column)

In [163]:
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [164]:
input_size = test_df_for_net.width - 1
output_size = 1

## Models instances 

In [165]:
def load_model(model_type, model_path, device, **model_kwargs):
    model = model_type(**model_kwargs)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

### RNN

In [166]:
hidden_size_rnn = input_size * 4
num_layers_rnn = 2

In [ ]:
rnn_kwargs = {
    "input_size": input_size,
    "hidden_size": hidden_size_rnn,
    "num_layers": num_layers_rnn,
    "output_size": output_size,
}

model_rnn = load_model(BasicRNN, "../models/rnn/best_model.pt", device, **rnn_kwargs)

In [168]:
y_true_rnn, y_pred_rnn = evaluate_model(model_rnn, test_loader, device)

In [ ]:
metrics_rnn = calculate_metrics(y_true_rnn, y_pred_rnn)
print(metrics_rnn)

{'MAPE': np.float32(9713.331), 'MAE': np.float32(10.957249)}


### LSTM

In [170]:
hidden_size_lstm = input_size * 4
num_layers_lstm = 2

In [171]:
lstm_kwargs = {
    "input_size": input_size,
    "hidden_size": hidden_size_lstm,
    "num_layers": num_layers_lstm,
    "output_size": output_size,
}

model_lstm = load_model(BasicLSTM, "../models/lstm/best_model.pt", device, **lstm_kwargs)

In [172]:
y_true_lstm, y_pred_lstm = evaluate_model(model_lstm, test_loader, device)

In [173]:
metrics_lstm = calculate_metrics(y_true_lstm, y_pred_lstm)
print(metrics_lstm)

{'MAPE': np.float32(10876.161), 'MAE': np.float32(10.943139)}


### BiLSTM

In [174]:
hidden_size_bi_lstm = input_size * 4
num_layers_bi_lstm = 2

In [175]:
bi_lstm_kwargs = {
    "input_size": input_size,
    "hidden_size": hidden_size_bi_lstm,
    "num_layers": num_layers_bi_lstm,
    "output_size": output_size,
}

model_bi_lstm = load_model(BasicBiLSTM, "../models/bi_lstm/best_model.pt", device, **bi_lstm_kwargs)

In [176]:
y_true_bi_lstm, y_pred_bi_lstm = evaluate_model(model_bi_lstm, test_loader, device)

In [177]:
metrics_bi_lstm = calculate_metrics(y_true_bi_lstm, y_pred_bi_lstm)
print(metrics_bi_lstm)

{'MAPE': np.float32(9199.742), 'MAE': np.float32(10.449373)}


### CNN LSTM

In [178]:
lstm_cnn_conv_filters = 9
lstm_cnn_kernel_size = 7
lstm_cnn_hidden_size = input_size * 4
lstm_cnn_num_layers = 2

In [179]:
cnn_lstm_kwargs = {
    "input_size": input_size,
    "conv_filters": lstm_cnn_conv_filters,
    "kernel_size": lstm_cnn_kernel_size,
    "lstm_hidden_size": lstm_cnn_hidden_size,
    "lstm_num_layers": lstm_cnn_num_layers,
    "output_size": output_size,
}

model_cnn_lstm = load_model(CNNLSTM, "../models/cnn_lstm/best_model.pt", device, **cnn_lstm_kwargs)

In [180]:
y_true_cnn_lstm, y_pred_cnn_lstm = evaluate_model(model_cnn_lstm, test_loader, device)

In [181]:
metrics_cnn_lstm = calculate_metrics(y_true_cnn_lstm, y_pred_cnn_lstm)
print(metrics_cnn_lstm)

{'MAPE': np.float32(9886.766), 'MAE': np.float32(10.619984)}


### Attention LSTM

In [182]:
hidden_size_attention_lstm = input_size * 4
num_layers_attention_lstm = 2

In [183]:
attention_lstm_kwargs = {
    "input_size": input_size,
    "hidden_size": hidden_size_attention_lstm,
    "num_layers": num_layers_attention_lstm,
    "attn_hidden_size": output_size,
    "output_size": output_size,
}

model_attention_lstm = load_model(AttentionLSTM, "../models/attention_lstm/best_model.pt", device, **attention_lstm_kwargs)

In [184]:
y_true_attention_lstm, y_pred_attention_lstm = evaluate_model(model_attention_lstm, test_loader, device)

In [185]:
metrics_attention_lstm = calculate_metrics(y_true_attention_lstm, y_pred_attention_lstm)
print(metrics_attention_lstm)

{'MAPE': np.float32(8839.676), 'MAE': np.float32(10.479506)}


Let's prepare test data for `Prophet` and `ARIMA`

In [186]:
df_full_ticks = pl.read_parquet("../data/processed/tx_blocks_eth_clean.parquet")

In [187]:
max_time = df_full_ticks["tx_time"].max()
test_threshold = max_time - timedelta(hours=1)
val_threshold = test_threshold - timedelta(hours=1)

test_threshold, val_threshold

(datetime.datetime(2025, 4, 13, 12, 43, 23),
 datetime.datetime(2025, 4, 13, 11, 43, 23))

In [188]:
df_blocks = df_full_ticks.group_by("block_hash").agg(pl.col("tx_time").min().alias("block_time"))

In [189]:
df = df_full_ticks.join(df_blocks, on="block_hash", how="left")

In [190]:
test_df = df.filter(pl.col("block_time") >= test_threshold)
test_df.shape

(75570, 20)

In [191]:
min_test_time = test_df["block_time"].min()
threshold = min_test_time + timedelta(minutes=5)

In [192]:
test_df = test_df.filter(pl.col("block_time") < threshold)
print(test_df.shape)

(7348, 20)


In [193]:
def add_pseudo_time(df: pd.DataFrame) -> pd.DataFrame:
    df_copy = df.copy(deep=True)

    df_copy["block_time"] = pd.to_datetime(df_copy["block_time"], unit="s")
    df_copy["block_timestamp"] = df_copy["block_time"].astype("int") // 10**6

    df_copy["n_in_block"] = df_copy.groupby("block_hash")["block_hash"].transform("count")
    df_copy["tx_index_in_block"] = df_copy.groupby("block_hash").cumcount()

    df_copy["tx_timestamp"] = df_copy["block_timestamp"] + df_copy["tx_index_in_block"] / df_copy["n_in_block"]

    df_copy["tx_time"] = pd.to_datetime(df_copy["tx_timestamp"], unit="s")
    
    df_copy = df_copy.drop(columns=[
        "block_timestamp", "tx_index_in_block", "n_in_block", "tx_timestamp"
    ])

    return df_copy

In [194]:
test_df_pd = test_df.select(["gas_price", "block_time", "block_hash"]).to_pandas()

test_df_pd = add_pseudo_time(test_df_pd)
test_df_pd.head()

,gas_price,block_time,block_hash,tx_time
0,-0.394365,2025-04-13 12:48:11,0xa2267aa8f7c058206cbda95258ac59dfe8639875e2b2...,2025-04-13 12:48:11.000000000
1,-0.368419,2025-04-13 12:48:11,0xa2267aa8f7c058206cbda95258ac59dfe8639875e2b2...,2025-04-13 12:48:11.003676414
2,-0.368403,2025-04-13 12:48:11,0xa2267aa8f7c058206cbda95258ac59dfe8639875e2b2...,2025-04-13 12:48:11.007352829
3,-0.382128,2025-04-13 12:48:11,0xa2267aa8f7c058206cbda95258ac59dfe8639875e2b2...,2025-04-13 12:48:11.011029482
4,-0.368403,2025-04-13 12:48:11,0xa2267aa8f7c058206cbda95258ac59dfe8639875e2b2...,2025-04-13 12:48:11.014705896


### Prophet

In [195]:
prophet_model = joblib.load("../models/prophet/prophet_model.pkl")

In [196]:
prophet_forecast = prophet_model.predict(test_df_pd[["tx_time"]].rename(columns={"tx_time": "ds"}))

In [197]:
y_true_prophet = test_df_pd["gas_price"].to_numpy()
y_pred_prophet = prophet_forecast["yhat"].to_numpy()

In [198]:
metrics_prophet = calculate_metrics(y_true_prophet, y_pred_prophet)
metrics_prophet

{'MAPE': np.float64(13346.916054122426), 'MAE': np.float64(14.432151873020732)}

### ARIMA

In [199]:
arima_model = joblib.load("../models/arima/arima_model.pkl")

In [200]:
arima_forecast = arima_model.forecast(steps=test_df_pd.shape[0])

In [201]:
y_true_arima = test_df_pd["gas_price"].to_numpy()
y_pred_arima = np.array(arima_forecast)

In [202]:
metrics_arima = calculate_metrics(y_true_arima, y_pred_arima)
metrics_arima

{'MAPE': np.float64(10845.863122073524), 'MAE': np.float64(12.963684445587745)}

## Final Comparison

In [ ]:
results = {
    "rnn": metrics_rnn,
    "lstm": metrics_lstm,
    "bi-lstm": metrics_bi_lstm,
    "cnn-lstm": metrics_cnn_lstm,
    "attention-lstm": metrics_attention_lstm,
    "prophet": metrics_prophet,
    "arima": metrics_arima
}

In [204]:
df_quality = pd.DataFrame(results).T
df_quality.index.name = "Model"

In [205]:
styled_df = df_quality.style.highlight_min(color="lightgreen", axis=0)
styled_df

,MAPE,MAE
Model,,
rnn,9713.331055,10.957249
lstm,10876.161133,10.943139
bi-lstm,9199.742188,10.449373
cnn-lstm,9886.765625,10.619984
attention-lstm,8839.675781,10.479506
prophet,13346.916054,14.432152
arima,10845.863122,12.963684
